In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/hh/MLops/dataScienceProject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str


In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])

    def get_model_trainer(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return model_trainer_config

In [8]:
import pandas as pd
import os
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression   # <-- better for HeartDisease (0/1)
# from sklearn.linear_model import ElasticNet         # if you insist on regression
from sklearn.metrics import accuracy_score
import pandas as pd, joblib, os

In [ ]:
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config=config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = train_data.drop([self.config.target_column], axis=1)
        train_y = test_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=43)
        logger.info(f"train set shape {train_x.shape}")
        logger.info(f"test set shape {test_x.shape}")
        logger.info(f"train set shape {train_y.shape}")
        logger.info(f"test set shape {test_y.shape}")
        
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [ ]:
try:
    obj = ConfigurationManager()
    model_trainer_config = obj.get_model_trainer()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.exception(e)
    raise e

[2025-08-10 14:02:56,754: INFO: common: yaml file config/config.yaml is loaded successfully]
[2025-08-10 14:02:56,756: INFO: common: yaml file params.yaml is loaded successfully]
[2025-08-10 14:02:56,760: INFO: common: yaml file schema.yaml is loaded successfully]
[2025-08-10 14:02:56,761: INFO: common: created directory at artifacts]
[2025-08-10 14:02:56,761: INFO: common: created directory at artifacts/model_trainer]
[2025-08-10 14:02:56,769: ERROR: 2287964262: Found input variables with inconsistent numbers of samples: [688, 230]]
Traceback (most recent call last):
  File "/var/folders/1z/knshtkp96gv94c7gjtb06jkh0000gn/T/ipykernel_8281/2287964262.py", line 5, in <module>
    model_trainer.train()
  File "/var/folders/1z/knshtkp96gv94c7gjtb06jkh0000gn/T/ipykernel_8281/4169583604.py", line 15, in train
    lr.fit(train_x, train_y)
  File "/Users/hh/MLops/dataScienceProject/venv/lib/python3.10/site-packages/sklearn/base.py", line 1365, in wrapper
    return fit_method(estimator, *args,

ValueError: Found input variables with inconsistent numbers of samples: [688, 230]